In [1]:
import pandas as pd

# Carregar os dataframes originais
# Certifique-se de que os arquivos CSV estão no mesmo diretório do script
# ou forneça o caminho completo para eles.
try:
    df_minamin_orig = pd.read_excel("Realtime_clean.xlsx")
    df_espelho_orig = pd.read_excel("Espelho BDES.xlsx")
    print("Arquivos originais carregados com sucesso!")
except FileNotFoundError as e:
    print(f"ERRO CRÍTICO: Arquivo não encontrado - {e.filename}")
    print("Por favor, verifique o nome e o local dos arquivos CSV de entrada.")
    df_minamin_orig = None # Garante que não prossiga se houver erro
    df_espelho_orig = None

# Prosseguir somente se ambos os dataframes foram carregados
if df_minamin_orig is not None and df_espelho_orig is not None:

    # --- 1. Expandir df_minamin para segundos ---
    print("Expandindo df_minamin para segundos...")
    df_minamin = df_minamin_orig.copy() # Trabalhar com uma cópia
    df_minamin['HORA_base'] = pd.to_datetime(df_minamin['HORA'], format='%H:%M:%S', errors='coerce')
    df_minamin.dropna(subset=['HORA_base'], inplace=True)

    minamin_segundos_list = []
    for _, row in df_minamin.iterrows():
        minuto_base = row['HORA_base']
        # Criar 60 segundos para este minuto
        for segundo_offset in range(60):
            new_row_data = row.to_dict()
            # Adiciona a data base '1900-01-01' para consistência no datetime
            new_row_data['datetime_segundo_audiencia'] = pd.to_datetime('1900-01-01 ' + (minuto_base + pd.Timedelta(seconds=segundo_offset)).strftime('%H:%M:%S'))
            minamin_segundos_list.append(new_row_data)

    df_minamin_segundos = pd.DataFrame(minamin_segundos_list)
    df_minamin_segundos.drop(columns=['HORA_base'], inplace=True, errors='ignore')
    df_minamin_segundos.sort_values('datetime_segundo_audiencia', inplace=True)
    print(f"df_minamin expandido para {len(df_minamin_segundos)} linhas (segundos).")

    # --- 2. Expandir df_espelho para segundos ---
    print("\nExpandindo df_espelho para segundos...")
    df_espelho = df_espelho_orig.copy() # Trabalhar com uma cópia
    df_espelho.rename(columns={'Início (Lauda)': 'inicio_lauda', 'Fim (Lauda)': 'fim_lauda'}, inplace=True)

    # Converte para time e depois adiciona a data base '1900-01-01'
    df_espelho['inicio_lauda_dt_time'] = pd.to_datetime(df_espelho['inicio_lauda'], format='%H:%M:%S', errors='coerce').dt.time
    df_espelho['fim_lauda_dt_time'] = pd.to_datetime(df_espelho['fim_lauda'], format='%H:%M:%S', errors='coerce').dt.time

    df_espelho.dropna(subset=['inicio_lauda_dt_time', 'fim_lauda_dt_time'], inplace=True)

    df_espelho['datetime_inicio_espelho_seg'] = pd.to_datetime('1900-01-01 ' + df_espelho['inicio_lauda_dt_time'].astype(str))
    df_espelho['datetime_fim_espelho_seg'] = pd.to_datetime('1900-01-01 ' + df_espelho['fim_lauda_dt_time'].astype(str))

    # Garantir que fim >= inicio
    df_espelho = df_espelho[df_espelho['datetime_fim_espelho_seg'] >= df_espelho['datetime_inicio_espelho_seg']]

    espelho_segundos_list = []
    for _, row in df_espelho.iterrows():
        segundos_no_intervalo = pd.date_range(row['datetime_inicio_espelho_seg'], row['datetime_fim_espelho_seg'], freq='S')
        
        for segundo_ts in segundos_no_intervalo:
            new_row_data = row.to_dict()
            new_row_data['datetime_segundo_espelho'] = segundo_ts
            espelho_segundos_list.append(new_row_data)

    if espelho_segundos_list:
        df_espelho_segundos = pd.DataFrame(espelho_segundos_list)
        df_espelho_segundos.drop(columns=['inicio_lauda_dt_time', 'fim_lauda_dt_time', 
                                          'datetime_inicio_espelho_seg', 'datetime_fim_espelho_seg'], 
                                 inplace=True, errors='ignore')
        df_espelho_segundos.sort_values('datetime_segundo_espelho', inplace=True)
        df_espelho_segundos.drop_duplicates(subset=['datetime_segundo_espelho'], keep='first', inplace=True)
        print(f"df_espelho expandido para {len(df_espelho_segundos)} linhas (segundos).")
    else:
        print("Não foi possível expandir df_espelho. Verifique os dados de início/fim.")
        df_espelho_segundos = pd.DataFrame(columns=['datetime_segundo_espelho', 'Bloco', 'Tipo', 'Retranca', 'Ordem', 'inicio_lauda', 'fim_lauda'])


    # --- 3. Merge em Nível de Segundo ---
    print("\nRealizando merge em nível de segundo...")
    df_merged_segundos = pd.merge(
        df_minamin_segundos,
        df_espelho_segundos,
        left_on='datetime_segundo_audiencia',
        right_on='datetime_segundo_espelho',
        how='left'
    )
    df_merged_segundos.drop(columns=['datetime_segundo_espelho'], inplace=True, errors='ignore')
    print(f"Merge finalizado. Total de {len(df_merged_segundos)} linhas.")

    # --- 4. Salvar ---
    output_path_segundos = "audiencia_espelho_merged_segundos.csv"
    df_merged_segundos.to_csv(output_path_segundos, index=False, encoding='utf-8-sig')
    print(f"\nArquivo mergeado em segundos salvo em: {output_path_segundos}")

    print("\nInformações do df_merged_segundos:")
    df_merged_segundos.info(verbose=False) # verbose=False para um resumo mais curto
else:
    print("\nUm ou ambos os DataFrames de entrada não foram carregados. O script não pode continuar.")

Arquivos originais carregados com sucesso!
Expandindo df_minamin para segundos...
df_minamin expandido para 9660 linhas (segundos).

Expandindo df_espelho para segundos...
df_espelho expandido para 8367 linhas (segundos).

Realizando merge em nível de segundo...
Merge finalizado. Total de 9660 linhas.


C:\Temp\ipykernel_16092\2242554504.py:59: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  segundos_no_intervalo = pd.date_range(row['datetime_inicio_espelho_seg'], row['datetime_fim_espelho_seg'], freq='S')



Arquivo mergeado em segundos salvo em: audiencia_espelho_merged_segundos.csv

Informações do df_merged_segundos:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9660 entries, 0 to 9659
Columns: 13 entries, HORA to Ordem
dtypes: datetime64[ns](1), float64(6), object(6)
memory usage: 981.2+ KB
